In [10]:
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import callbacks
from sklearn.preprocessing import LabelEncoder
import numpy as np
import h5py
import os
import datetime
import time

#import effTools as eff
#!pip install pandas
import pandas as pd
from keras.models import load_model
import json
import tensorflow as tf
print(tf.__version__)

1.14.0


In [11]:
datasets_available = ["Training","Testing"]
dataset = datasets_available
img_width, img_height = 150, 150
image_size = (150, 150)
nb_train_samples = 2000
nb_validation_samples = 800
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 2 #change
batch_size = 32
lr = 0.0004


In [12]:
import cv2
for ds in range(0, len(dataset)):
    model_name    = "preprocesss"
    weights       = "imagenet"
    include_top   = 0 
    train_path    = "preprocess/" + dataset[ds]
    features_path = "output/" +dataset[ds]+ "/features.h5" 
    labels_path   = "output/" +dataset[ds]+ "/labels.h5"

    #test_size     = test_set[ds]
    
    # check if the output directory exists, if not, create it.
    #eff.check_if_directory_exists("output")
    
    # check if the output directory exists, if not, create it.
    #eff.check_if_directory_exists("output/" + dataset[ds])
    
    # start time
    print ("[STATUS] start time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
    start = time.time()
    
    if ds==0:
        img_width, img_height = 630,630
        image_size = (630, 630)
        nb_train_samples = 2000
        nb_validation_samples = 800
        nb_filters1 = 32
        nb_filters2 = 64
        nb_filters3 = 128
        conv1_size = 3
        conv2_size = 2
        conv3_size = 2
        pool_size = 2
        classes_num = 2 #change
        batch_size = 32
        lr = 0.0004

        model = Sequential()
        model.add(Conv2D(nb_filters1, (conv1_size, conv1_size), padding="same", input_shape=(img_width, img_height, 3)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

        model.add(Conv2D(nb_filters2, (conv2_size, conv2_size), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(pool_size, pool_size), data_format='channels_first'))
        
        model.add(Conv2D(nb_filters3, (conv3_size, conv3_size), padding="same"))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(pool_size, pool_size), data_format='channels_first'))
        
        model.summary()
   
        
    # check if the output directory exists, if not, create it.
    #eff.check_if_directory_exists("output/" + dataset[ds] + "/" + model_name)
    
    print ("[INFO] successfully loaded base model and model...")
    
    # path to training dataset
    train_labels = os.listdir(train_path)
    
    # encode the labels
    print ("[INFO] encoding labels...")
    le = LabelEncoder()
    le.fit([tl for tl in train_labels])
    
    # variables to hold features and labels
    features = []
    labels   = []
 
    count = 1
    for i, label in enumerate(train_labels):   
      cur_path = train_path+ "/" + label
       # check how many files are, together with their extensions
      #print(cur_path)
      list_files = os.listdir(cur_path)  
      count = 1
      #for image_path in glob.glob(cur_path + "/*.jpg"):
      for image_path in range(0, len(list_files)):
        img = cv2.imread(cur_path + "/" + list_files[image_path]) 
        img = cv2.resize(img, (630,630)) 
        cv2.imwrite('1.jpg', img)
        #print ("[INFO] Processing - " + str(count) + " named " + list_files[image_path])
        img = image.load_img('1.jpg')
        
        x = image.img_to_array(img)
        
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        feature = model.predict(x)
        flat = feature.flatten()
        features.append(flat)
        #print(flat)
        
        labels.append(label)
        #print ("[INFO] processed for image - " + list_files[image_path])
        count += 1
      print ("[INFO] completed label - " + label)
    
    imgs = cv2.imread('1.jpg')    
    img_gray = cv2.cvtColor(imgs, cv2.COLOR_BGR2GRAY)
    r, threshold = cv2.threshold(img_gray, 125, 255, cv2.THRESH_BINARY)
    edged = cv2.Canny(img_gray, 100,200)
    contours, hierarchy = cv2.findContours(edged,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  
    cv2.drawContours(img_gray, contours, -1, (0, 255, 0), 3)
    # encode the labels using LabelEncoder
    le = LabelEncoder()
    le_labels = le.fit_transform(labels)
    
    # get the shape of training labels
    print ("[STATUS] training labels: {}".format(le_labels))
    print ("[STATUS] training labels shape: {}".format(le_labels.shape))
    
    # save features and labels
    try:
        h5f_data = h5py.File(features_path, 'w')
    except:
        a=1;
        
    h5f_data.create_dataset('dataset_1', data=np.array(features))
    
    h5f_label = h5py.File(labels_path, 'w')
    h5f_label.create_dataset('dataset_1', data=np.array(le_labels))
    
    h5f_data.close()
    h5f_label.close()
    print('ok')
    if ds==0:
        model.save('output/model1.h5')
        model_json = model.to_json()
        with open('output/model1.json', "w") as json_file:
            json_file.write(model_json)
        model.save_weights('output/weights1.h5')

[STATUS] start time - 2022-05-27 12:33
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 630, 630, 32)      896       
_________________________________________________________________
activation_4 (Activation)    (None, 630, 630, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 315, 315, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 315, 315, 64)      8256      
_________________________________________________________________
activation_5 (Activation)    (None, 315, 315, 64)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 315, 157, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 3

In [4]:
from __future__ import print_function

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import svm
import numpy as np
import h5py
import os
import json
import pickle
!pip install matplotlib
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
h5f_datatrain  = h5py.File('output/Training/features.h5', 'r')
h5f_labeltrain = h5py.File('output/Training/labels.h5', 'r')
h5f_datatest  = h5py.File('output/Testing/features.h5', 'r')
h5f_labeltest = h5py.File('output/Testing/labels.h5', 'r')

features_stringtrain = h5f_datatrain['dataset_1']
labels_stringtrain   = h5f_labeltrain['dataset_1']
features_stringtest = h5f_datatest['dataset_1']
labels_stringtest   = h5f_labeltest['dataset_1']

featurestrain = np.array(features_stringtrain)
labelstrain   = np.array(labels_stringtrain)

featurestest = np.array(features_stringtest)
labelstest   = np.array(labels_stringtest)

h5f_datatrain.close()
h5f_labeltrain.close()
h5f_datatest.close()
h5f_labeltest.close()

In [6]:
print ("[INFO] train data: {}".format(featurestrain.shape))
print ("[INFO] train label: {}".format(labelstrain.shape))
print ("[INFO] test data: {}".format(featurestest.shape))
print ("[INFO] test label: {}".format(labelstest.shape))

[INFO] train data: (253, 1572480)
[INFO] train label: (253,)
[INFO] test data: (253, 1572480)
[INFO] test label: (253,)


In [7]:
from keras.preprocessing import image
import numpy as np
from keras.models import model_from_json
#from deeplearning import graph, output_list
import cv2
import time
import joblib
#from sklearn.externals import joblib 
#import pymysql as mdb
print('k')
from keras.applications.inception_v3 import InceptionV3, preprocess_input
json_file = open('output/model1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("output/weights1.h5")
print("Loaded model from disk")

k
Loaded model from disk


In [8]:

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
import joblib
print ("[STATUS] start time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
neigh = KNeighborsClassifier(n_neighbors=3)
filename = 'knn_model.sav'
joblib.dump(neigh, filename)
neigh.fit(featurestrain, labelstrain)
preds = neigh.predict(featurestest)
print(classification_report(labelstest, preds))
joblib.dump(neigh, filename)


print('train complete')
print ("[STATUS] End Time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

[STATUS] start time - 2022-05-27 12:04
              precision    recall  f1-score   support

           0       0.80      0.90      0.85        98
           1       0.93      0.86      0.89       155

    accuracy                           0.87       253
   macro avg       0.87      0.88      0.87       253
weighted avg       0.88      0.87      0.87       253

train complete
[STATUS] End Time - 2022-05-27 12:05


In [9]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
import joblib
print ("[STATUS] start time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
neigh = SVC()
filename = 'svm_model.sav'
joblib.dump(neigh, filename)
neigh.fit(featurestrain, labelstrain)
preds = neigh.predict(featurestrain)
print(classification_report(labelstrain, preds))
joblib.dump(neigh, filename)


print('train complete')
print ("[STATUS] End time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))

[STATUS] start time - 2022-05-27 09:33
              precision    recall  f1-score   support

           0       0.85      0.72      0.78        98
           1       0.84      0.92      0.88       155

    accuracy                           0.84       253
   macro avg       0.84      0.82      0.83       253
weighted avg       0.84      0.84      0.84       253

train complete
[STATUS] End time - 2022-05-27 09:45


In [16]:
import matplotlib.pyplot as plt
from PIL import Image
from random import randint

##myfile = 'oo.jpg'
##imag=cv2.imread(myfile)
##imag=cv2.resize(imag, (630,630))
a=['Benign','Malignant']          
!pip install pymysql

from keras.preprocessing import image
import os
from keras.preprocessing import image
import numpy as np
##from deeplearning import graph, model, output_list
import cv2
import requests
import pymysql as mdb

import time
def upd(disease, ids):
    
    mydb = mdb.connect(
      host="127.0.0.1",
      user="root",
      passwd="",
      database="370project"
    )

    mycursor = mydb.cursor()

   

    sql = "UPDATE disease SET disease = %s WHERE ids = %s"
    val = (disease, ids)

    mycursor.execute(sql, val)

    mydb.commit()
    mydb.close()
while True:
    time.sleep(2)
    print('ok')
    mydb = mdb.connect(
          host="127.0.0.1",
          user="root",
          passwd="",
          database="370project"
        )

    mycursor = mydb.cursor()

    sql = "SELECT ids,pic,disease FROM disease"


    mycursor.execute(sql)
    fine=0
    myresult = mycursor.fetchall()
    mydb.close()
    #print (myresult)
    for x in myresult:
        
      ids= str(x[0])
      pic=str(x[1])
      fl=str(x[2])
      #print (ids,pic,fl)
      if fl=='1': 
        print('ok')
        print(pic)
        pic='D:/xampp/htdocs/imgsent/process/images/'+pic
        myfile = pic    
        img = cv2.imread(myfile) 
        img = cv2.resize(img, (630,630)) 
        cv2.imwrite('1.jpg', img)
        #print ("[INFO] Processing - " + str(count) + " named " + list_files[image_path])
        img = image.load_img('1.jpg')
        imgs = cv2.imread('1.jpg')    
        img_gray = cv2.cvtColor(imgs, cv2.COLOR_BGR2GRAY)
        r, threshold = cv2.threshold(img_gray, 125, 255, cv2.THRESH_BINARY)
        edged = cv2.Canny(img_gray, 100,200)
        contours, hierarchy = cv2.findContours(edged,cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)  
        cv2.drawContours(imgs, contours, -1, (0, 255, 0), 3) 
        
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = preprocess_input(img)        
        prediction = loaded_model.predict(img)
        prediction_flatten = prediction.flatten()
        
        knn_from_joblib = joblib.load('knn_model.sav')
        prediction_flatten=prediction_flatten.reshape(1,-1)
        max_val_index=knn_from_joblib.predict(prediction_flatten) 

        max_val_index=int(max_val_index)
        print (max_val_index)   
        
        pred='Tumor Is '
        disease=(pred+a[max_val_index])
        upd(disease, ids)

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
Y18.JPG
1
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok


KeyboardInterrupt: 